In [1]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import seaborn as sns
import bokeh
from bokeh.models import ColumnDataSource, FactorRange
from bokeh.plotting import figure
from bokeh.io import show, output_notebook, output_file
from bokeh.models import Legend

In [3]:
# Retrieve "Motor Vehicle Collisions - Crashes" data
url = 'https://data.cityofnewyork.us/api/views/h9gi-nx95/rows.csv?accessType=DOWNLOAD'
df = pd.read_csv(url)
print(df.shape)
df.head()

d:\programmerd\anaconda\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(1673457, 29)


,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,10/16/2016,14:00,BROOKLYN,11211,40.714878,-73.95447,POINT (-73.95447 40.714878),NORTH 6 STREET,HAVEMEYER STREET,NaN,...,Unspecified,NaN,NaN,NaN,3543015,PASSENGER VEHICLE,PICK-UP TRUCK,NaN,NaN,NaN
1,11/04/2016,9:30,NaN,NaN,NaN,NaN,NaN,WEST 168 STREET,RIVERSIDE DRIVE,NaN,...,Unspecified,NaN,NaN,NaN,3554750,SPORT UTILITY / STATION WAGON,PASSENGER VEHICLE,NaN,NaN,NaN
2,10/25/2016,15:33,BROOKLYN,11209,40.634620,-74.02842,POINT (-74.02842 40.63462),NaN,NaN,229 71 STREET,...,Unspecified,NaN,NaN,NaN,3547136,SPORT UTILITY / STATION WAGON,PASSENGER VEHICLE,NaN,NaN,NaN
3,11/06/2016,21:00,QUEENS,11416,40.682484,-73.86371,POINT (-73.86371 40.682484),75 STREET,95 AVENUE,NaN,...,Unspecified,Unspecified,Unspecified,NaN,3555199,SPORT UTILITY / STATION WAGON,PASSENGER VEHICLE,PASSENGER VEHICLE,PASSENGER VEHICLE,SPORT UTILITY / STATION WAGON
4,10/17/2016,4:00,BROOKLYN,11220,40.644455,-74.01830,POINT (-74.0183 40.644455),56 STREET,3 AVENUE,NaN,...,Unspecified,NaN,NaN,NaN,3541751,SPORT UTILITY / STATION WAGON,NaN,NaN,NaN,NaN


In [4]:
print(df.columns)

Index(['CRASH DATE', 'CRASH TIME', 'BOROUGH', 'ZIP CODE', 'LATITUDE',
       'LONGITUDE', 'LOCATION', 'ON STREET NAME', 'CROSS STREET NAME',
       'OFF STREET NAME', 'NUMBER OF PERSONS INJURED',
       'NUMBER OF PERSONS KILLED', 'NUMBER OF PEDESTRIANS INJURED',
       'NUMBER OF PEDESTRIANS KILLED', 'NUMBER OF CYCLIST INJURED',
       'NUMBER OF CYCLIST KILLED', 'NUMBER OF MOTORIST INJURED',
       'NUMBER OF MOTORIST KILLED', 'CONTRIBUTING FACTOR VEHICLE 1',
       'CONTRIBUTING FACTOR VEHICLE 2', 'CONTRIBUTING FACTOR VEHICLE 3',
       'CONTRIBUTING FACTOR VEHICLE 4', 'CONTRIBUTING FACTOR VEHICLE 5',
       'COLLISION_ID', 'VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2',
       'VEHICLE TYPE CODE 3', 'VEHICLE TYPE CODE 4', 'VEHICLE TYPE CODE 5'],
      dtype='object')


In [18]:
# Convert date column to datetime
df['CRASH DATE'] = pd.to_datetime(df['CRASH DATE']) 

In [188]:
# Set date interval
start_date = '2019-04-25'
end_date = '2020-04-25'

interval_data = df.copy()

# Change df to only include data for the dates within the interval
interval = (interval_data['CRASH DATE'] > start_date) & (interval_data['CRASH DATE'] <= end_date)

# The interval variable is used to set the new dates in the 
# interval_data DataFrame, so that only the ones with interval=true
# will be used.
interval_data = interval_data.loc[interval]

In [189]:
print(df.shape, interval_data.shape)

(1673457, 29) (188051, 29)


In [190]:
# Create dataframe to use for bokeh plots, containing number of pedestrians, 
# cyclists, and motorists injured or killed in different time series
time_series_df = interval_data[['CRASH DATE', 'CRASH TIME', 'NUMBER OF PERSONS INJURED',
       'NUMBER OF PERSONS KILLED', 'NUMBER OF PEDESTRIANS INJURED',
       'NUMBER OF PEDESTRIANS KILLED', 'NUMBER OF CYCLIST INJURED',
       'NUMBER OF CYCLIST KILLED', 'NUMBER OF MOTORIST INJURED',
       'NUMBER OF MOTORIST KILLED']].copy()

In [191]:
# Create column to store hour of day
time_series_df['HOUR OF DAY'] = pd.to_datetime(time_series_df['CRASH TIME']).dt.hour.copy()

In [192]:
# Create column to store day of the week
time_series_df['WEEKDAY'] = pd.to_datetime(time_series_df['CRASH DATE']).dt.dayofweek.copy()

In [193]:
# Create column to store hour of the week
time_series_df['HOUR OF WEEK'] = time_series_df['WEEKDAY'] * 24 + time_series_df['HOUR OF DAY']

In [194]:
# Create column to store month 
time_series_df['MONTH'] = pd.to_datetime(time_series_df['CRASH DATE']).dt.month.copy()

In [195]:
# Merge number of injured and dead for pedestrians, persons, cyclists, and motorists
time_series_df['PEDESTRIANS'] = time_series_df['NUMBER OF PEDESTRIANS INJURED'] + time_series_df['NUMBER OF PEDESTRIANS KILLED']
time_series_df['PERSONS'] = time_series_df['NUMBER OF PERSONS INJURED'] + time_series_df['NUMBER OF PERSONS KILLED']
time_series_df['CYCLISTS'] = time_series_df['NUMBER OF CYCLIST INJURED'] + time_series_df['NUMBER OF CYCLIST KILLED']
time_series_df['MOTORISTS'] = time_series_df['NUMBER OF MOTORIST INJURED'] + time_series_df['NUMBER OF MOTORIST KILLED']

In [196]:
# Drop not needed columns
time_series_df = time_series_df.drop(['CRASH DATE', 'CRASH TIME', 'NUMBER OF PERSONS INJURED',
       'NUMBER OF PERSONS KILLED', 'NUMBER OF PEDESTRIANS INJURED',
       'NUMBER OF PEDESTRIANS KILLED', 'NUMBER OF CYCLIST INJURED',
       'NUMBER OF CYCLIST KILLED', 'NUMBER OF MOTORIST INJURED',
       'NUMBER OF MOTORIST KILLED'], axis=1)

In [197]:
# Reset indices after drop
time_series_df.reset_index(drop=True, inplace = True)

In [198]:
# The PERSON column is of type float, change to integer
time_series_df['PERSONS'] = time_series_df['PERSONS'].astype(int)
time_series_df.head()

,HOUR OF DAY,WEEKDAY,HOUR OF WEEK,MONTH,PEDESTRIANS,PERSONS,CYCLISTS,MOTORISTS
0,0,0,0,4,0,0,0,0
1,2,0,2,4,0,1,0,1
2,15,5,135,4,0,1,0,1
3,18,5,138,4,0,0,0,0
4,5,5,125,4,0,0,0,0


In [199]:
# Make base dataframe to store data for plots
# Make list to hold names for accidents
accident_types = list(time_series_df.columns[4:])
base_data = pd.DataFrame(columns = list(accident_types))

base_data.head()

,PEDESTRIANS,PERSONS,CYCLISTS,MOTORISTS


In [228]:
# Hour of day
hour_of_day_df = base_data.copy()
unique_hours = time_series_df['HOUR OF DAY'].unique().tolist()
unique_hours.sort()
# Range hours from 1-24 instead of 0-23 
unique_hours = [x+1 for x in unique_hours]
hour_of_day_df['HOUR OF DAY'] = unique_hours

# Days in week
day_in_week_df = base_data.copy()
unique_days = time_series_df['WEEKDAY'].unique().tolist()
unique_days.sort()
day_in_week_df['WEEKDAY'] = unique_days

# Hours in week
hour_of_week_df = base_data.copy()
unique_hours_week = time_series_df['HOUR OF WEEK'].unique().tolist()
unique_hours_week.sort()
# Range hours from 1-24 instead of 0-23 
unique_hours_week = [x+1 for x in unique_hours_week]
hour_of_week_df['HOUR OF WEEK'] = unique_hours_week

# Month in year
month_in_year_df = base_data.copy()
unique_months = time_series_df['MONTH'].unique().tolist()
unique_months.sort()
month_in_year_df['MONTH'] = unique_months


for accident_type in range(len(accident_types)):
    total_num_accidents_for_type = time_series_df[accident_types[accident_type]].sum()
    temp_hour_of_day_accident_type_df = time_series_df[['HOUR OF DAY', accident_types[accident_type]]].copy()
    temp_day_in_week_accident_type_df = time_series_df[['WEEKDAY', accident_types[accident_type]]].copy()
    temp_hour_in_week_accident_type_df = time_series_df[['HOUR OF WEEK', accident_types[accident_type]]].copy()
    temp_month_accident_type_df = time_series_df[['MONTH', accident_types[accident_type]]].copy()

    for hour in unique_hours:
        temp_df_hour = temp_hour_of_day_accident_type_df[temp_hour_of_day_accident_type_df['HOUR OF DAY'] == hour]
        num_accidents_for_hour = temp_df_hour[accident_types[accident_type]].sum()
        hour_of_day_df.iloc[hour-1,accident_type] = num_accidents_for_hour / total_num_accidents_for_type
    
    for day in unique_days:
        temp_df_day = temp_day_in_week_accident_type_df[temp_day_in_week_accident_type_df['WEEKDAY'] == day]
        num_accidents_for_day = temp_df_day[accident_types[accident_type]].sum()
        day_in_week_df.iloc[day,accident_type] = num_accidents_for_day / total_num_accidents_for_type
        
    for hour in unique_hours_week:
        temp_df_hour = temp_hour_in_week_accident_type_df[temp_hour_in_week_accident_type_df['HOUR OF WEEK'] == hour]
        num_accidents_for_hour = temp_df_hour[accident_types[accident_type]].sum()
        hour_of_week_df.iloc[hour-1,accident_type] = num_accidents_for_hour / total_num_accidents_for_type
        
    for month in unique_months:
        temp_df_month = temp_month_accident_type_df[temp_month_accident_type_df['MONTH'] == month]
        num_accidents_for_month = temp_df_month[accident_types[accident_type]].sum()
        month_in_year_df.iloc[month-1,accident_type] = num_accidents_for_month / total_num_accidents_for_type

# Make string of unique hours, days, hours in week, and months
str_unique_hours = [str(x) for x in unique_hours]
str_unique_hours.append('')
str_unique_days = [str(x) for x in unique_days]
str_unique_days.append('')
str_unique_hours_week = [str(x) for x in unique_hours_week]
str_unique_hours_week.append('')
str_unique_months = [str(x) for x in unique_months]
str_unique_months.append('')

sources = []
hour_of_day_source = ColumnDataSource(hour_of_day_df)
sources.append(hour_of_day_source)
day_in_week_source = ColumnDataSource(day_in_week_df)
sources.append(day_in_week_source)
hour_of_week_source = ColumnDataSource(hour_of_week_df)
sources.append(hour_of_week_source)
month_in_year_source = ColumnDataSource(month_in_year_df)
sources.append(month_in_year_source)

In [229]:
# Prepare for Bokeh Plots
# Call output_notebook() to display bokeh in the notebook
output_notebook()

Loading BokehJS ...

In [230]:
palette = sns.color_palette("hls", len(accident_types))
pal = palette.as_hex()

ps = []

# Plot for "HOUR IN DAY"
p_hour_in_day = figure(plot_height = 400, plot_width = 850, 
           x_range = FactorRange(factors=str_unique_hours),
           title = 'Accidents for every Hour of the Day', 
           x_axis_label = 'Hour of the Day',
           y_axis_label = 'Relative Frequency')
ps.append(p_hour_in_day)

# Plot for "DAYS IN WEEK"
p_days_in_week = figure(plot_height = 400, plot_width = 850, 
           x_range = FactorRange(factors=str_unique_days),
           title = 'Accidents for every Day of the Week', 
           x_axis_label = 'Day of the Week',
           y_axis_label = 'Relative Frequency')
ps.append(p_days_in_week)

# Plot for "HOUR IN WEEK"
p_hour_in_week = figure(plot_height = 400, plot_width = 850, 
           x_range = FactorRange(factors=str_unique_hours_week),
           title = 'Accidents for every Hour of the Week', 
           x_axis_label = 'Hour of the Week',
           y_axis_label = 'Relative Frequency')
ps.append(p_hour_in_week)

# Plot for "MONTHS IN YEAR"
p_month_in_year = figure(plot_height = 400, plot_width = 850, 
           x_range = FactorRange(factors=str_unique_months),
           title = 'Accidents for every Month of the Year', 
           x_axis_label = 'Month of the Year',
           y_axis_label = 'Relative Frequency')
ps.append(p_month_in_year)
xs = ('HOUR OF DAY', 'WEEKDAY', 'HOUR OF WEEK', 'MONTH')

for j, p in enumerate(ps):
    bar = {}
    items = [] 

    for indx, i in enumerate(accident_types):
        bar[i] = p.vbar(x=xs[j], 
                        muted_alpha=0.03, 
                        fill_alpha=0.7,  
                        line_color=(0,0,0,0.0), 
                        muted=True, 
                        width=0.6, 
                        top=i, 
                        color=pal[indx], 
                        muted_color=pal[indx], 
                        source=sources[j])
        items.append((i, [bar[i]])) 

    legend = Legend(items=items, location=(0,220)) 
    p.add_layout(legend, 'left')
    p.legend.click_policy="mute"

In [227]:
for p in ps:
    show(p)

Weekly (monday, tuesday…)
Monthly (january, february…)
Hour in day (0-23)
Weekly and hourly (168 hours during week)
